### Цели на машинное обучение
В последующей работе мы сфокусируемся на анализе успешности прошедших проектов. В связи с этим у нас есть две задачи машинного обучения:

**Основная**
1. На основе только успешных проектов мы создадим модель, предсказывающую сколько денег собирает тот или иной проект в зависимости от признаков. Кол-во собранных денег - один из основных показателей успешности проекта, помимо самого факта достижения желанной суммы.

**Дополнительная**

2. Предсказать среднее вложение в каждый проект. Это задача будет выполняться для всех собранных данных без учета колонок days, rate и money. Колонка money будет использована для вычисления новых средних вложений в проект (переменная money/supp). По оставшимся данным мы будем предсказывать средние вложения для проектов.



In [684]:
# импортируем бибилиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
from nltk.tokenize import word_tokenize
import nltk
from nltk import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.notebook import tqdm

from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error

# Отключение некоторых лишних предупреждений
import warnings
warnings.filterwarnings("ignore")

In [685]:
nltk.download("punkt", quiet=True) # модуль для обработки текстовых данных

True

In [686]:
df = pd.read_pickle('df_hyp.pkl') # считываем датафрейм
df

,href,title,description,location,tags,upd,comm,supp,rewards_num,money,...,tags_Transport,tags_Travel,tags_University,tags_Venue,tags_null,sentiment_score,ps_capital,inn_score,eco_score,pledge
0,https://www.crowdfunder.co.uk/free-assange,Help campaign to Free Julian Assange,"[help, campaign, free, julian, assange, stop, ...",London,"[Community, Personal Causes]",22,1242,3576,0,300000.0,...,0,0,0,0,0,0.7840,0,0,0,83.892617
1,https://www.crowdfunder.co.uk/blackout2023,Black Out 2023 | Cannes Lions Festival,"[taking, black, talent, connected, creative, i...",London,"[Business, Music]",0,9,24,0,100000.0,...,0,0,0,0,0,0.9217,0,2,0,4166.666667
2,https://www.crowdfunder.co.uk/50-days-to-make-...,50 Days to Make a Difference,"[scientists, curate, portfolio, effective, cli...",London,[Environment],0,11,68,3,150000.0,...,0,0,0,0,0,0.7430,1,0,2,2205.882353
3,https://www.crowdfunder.co.uk/saveside,#SAVESIDE,"[april, side, gallery, close, public, due, cri...",Newcastle upon Tyne,[Creative & Arts],5,694,1806,0,75000.0,...,0,0,0,0,0,-0.7096,0,0,0,41.528239
4,https://www.crowdfunder.co.uk/lets-smash-the-p...,Let's smash the political silence on Brexit,"[everyone, knows, brexit, working, politicians...",London,[Politics],1,407,2391,6,100000.0,...,0,0,0,0,0,0.0000,0,0,0,41.823505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1947,https://www.crowdfunder.co.uk/schumacher,Schumacher International Network for Change (S...,"[raising, days, build, virtual, campus, connec...",Totnes,null,2,27,200,8,25225.0,...,0,0,0,0,1,0.5106,0,0,0,126.125000
1948,https://www.crowdfunder.co.uk/ipcuk-scholarships,IPCUK Scholarships,"[help, build, scholarship, fund, support, prac...",London,null,5,26,238,27,24255.0,...,0,0,0,0,1,0.6597,0,0,0,101.911765
1949,https://www.crowdfunder.co.uk/snaffling-pig,Snaffling Pig,"[porky, brand, balls, take, snack, market, inn...",United Kingdom,null,2,0,92,13,23600.0,...,0,0,0,0,1,0.7184,1,1,0,256.521739
1950,https://www.crowdfunder.co.uk/hookpod-technolo...,Hookpod; technology to save seabirds and turtles,"[work, make, longline, fishing, safe, marine, ...",Dartington,null,14,42,487,8,72997.0,...,0,0,0,0,1,0.4404,0,0,0,149.891170


In [687]:
df.drop('description', axis=1, inplace= True)

In [688]:
df1 =pd.read_pickle('df_main.pkl') # считываем датафрейм для обработки описаний
df1

,href,title,description,location,tags,upd,comm,supp,rewards_num,money,rate,days,status
0,https://www.crowdfunder.co.uk/free-assange,Help campaign to Free Julian Assange,Help campaign to Free Julian Assange and Stop ...,"London, Greater London, United Kingdom","[Community, Personal Causes]",22,1242,3576,0,300000,0.61,null,funding
1,https://www.crowdfunder.co.uk/blackout2023,Black Out 2023 | Cannes Lions Festival,Taking Black talent connected to the creative ...,"London, Greater London, United Kingdom","[Business, Music]",0,9,24,0,100000,0.84,null,funding
2,https://www.crowdfunder.co.uk/50-days-to-make-...,50 Days to Make a Difference,Our scientists curate a portfolio of effective...,"London, Greater London, United Kingdom",[Environment],0,11,68,3,150000,0.55,null,funding
3,https://www.crowdfunder.co.uk/saveside,#SAVESIDE,"As of 9th April 2023, Side Gallery will close ...","Newcastle upon Tyne, Tyne and Wear, United Kin...",[Creative & Arts],5,694,1806,0,75000,1.04,null,funding
4,https://www.crowdfunder.co.uk/lets-smash-the-p...,Let's smash the political silence on Brexit,"Everyone knows Brexit isn't working, but polit...","London, Greater London, United Kingdom",[Politics],1,407,2391,6,100000,0.6,null,funding
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1947,https://www.crowdfunder.co.uk/schumacher,Schumacher International Network for Change (S...,We are raising £25K in 25 days to build a virt...,Totnes,null,2,27,200,8,25225,null,25,success
1948,https://www.crowdfunder.co.uk/ipcuk-scholarships,IPCUK Scholarships,Help build a scholarship fund so that we can s...,London,null,5,26,238,27,24255,null,28,success
1949,https://www.crowdfunder.co.uk/snaffling-pig,Snaffling Pig,A porky brand with the balls to take on the sn...,,null,2,0,92,13,23600,null,42,success
1950,https://www.crowdfunder.co.uk/hookpod-technolo...,Hookpod; technology to save seabirds and turtles,We work to make longline fishing safe for mari...,"Dartington, England, United Kingdom",null,14,42,487,8,72997,null,10,success


In [689]:
df1.description.fillna('Liberty is raising £18,000 to defend human rights in the UK and make millions aware of threats to our rights. Donate now to help us reach our target', inplace=True)

Мы попробовали включить в модель текстовые признаки напрямую.

In [690]:
df1["description"].dropna(inplace=True)

# convert the column to string
df1['description'] = df1['description'].astype(str)

# apply the `word_tokenize()` function
tokens = df1['description'].apply(word_tokenize)

In [691]:
desc = df1[['description']]
desc

,description
0,Help campaign to Free Julian Assange and Stop ...
1,Taking Black talent connected to the creative ...
2,Our scientists curate a portfolio of effective...
3,"As of 9th April 2023, Side Gallery will close ..."
4,"Everyone knows Brexit isn't working, but polit..."
...,...
1947,We are raising £25K in 25 days to build a virt...
1948,Help build a scholarship fund so that we can s...
1949,A porky brand with the balls to take on the sn...
1950,We work to make longline fishing safe for mari...


In [692]:
df2 = df.join(desc)

Оставим датафрейм без выбросов.

In [779]:
df4 = df2[(df2['supp'] < 4000) & (df2['rewards_num'] < 60) & (df2['money'] < 60000)]
df4

,href,title,location,tags,upd,comm,supp,rewards_num,money,status,...,tags_Travel,tags_University,tags_Venue,tags_null,sentiment_score,ps_capital,inn_score,eco_score,pledge,description
6,https://www.crowdfunder.co.uk/new-golf-gym---l...,New Golf Gym - Llantrisant and Pontyclun juniors,Pontyclun,"[Community, Sports]",2,44,117,5,52055.000000,funding,...,0,0,0,0,0.7712,0,2,0,444.914530,We want to create an inclusive new gym and loc...
7,https://www.crowdfunder.co.uk/the-destiny-camp...,The Destiny Campus Community Project,Southampton,"[Community, Business]",0,66,160,0,53323.000000,funding,...,0,0,0,0,-0.1027,0,0,0,333.268750,Serving our community with love; bridging pove...
9,https://www.crowdfunder.co.uk/food-9,Eat More for Less,London,"[Food and Drink, Social Enterprise]",0,58,101,0,50000.000000,funding,...,0,0,0,0,0.6705,0,0,0,495.049505,Helping families make budget friendly choices ...
10,https://www.crowdfunder.co.uk/csaa-food-bank,# Stop Hunger! Food and Support For People in ...,London,"[Food and Drink, Community]",0,38,231,0,45258.000000,funding,...,0,0,0,0,-0.5719,0,0,0,195.922078,We are raising money to provide a food parcels...
11,https://www.crowdfunder.co.uk/crowdfunder-cost...,Cost Of Living Emergency Fund,United Kingdom,"[Community, Charities]",5,60,735,0,38976.294424,funding,...,0,0,0,0,0.4019,0,0,0,53.028972,Millions of UK households are facing rising li...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1945,https://www.crowdfunder.co.uk/on-london-co-uk,OnLondon.co.uk,London,null,7,169,434,13,25401.000000,success,...,0,0,0,1,0.5719,1,0,0,58.527650,To keep a dedicated news and comment website a...
1946,https://www.crowdfunder.co.uk/titseybrewingco,Help Titsey Brewing Co reach new beery heights,Warlingham,null,4,141,269,16,25275.000000,success,...,0,0,0,1,0.2716,0,1,0,93.959108,Titsey Brewing Co is struggling to meet demand...
1947,https://www.crowdfunder.co.uk/schumacher,Schumacher International Network for Change (S...,Totnes,null,2,27,200,8,25225.000000,success,...,0,0,0,1,0.5106,0,0,0,126.125000,We are raising £25K in 25 days to build a virt...
1948,https://www.crowdfunder.co.uk/ipcuk-scholarships,IPCUK Scholarships,London,null,5,26,238,27,24255.000000,success,...,0,0,0,1,0.6597,0,0,0,101.911765,Help build a scholarship fund so that we can s...


Оставим успешные проекты. Делаем первую задачу.

In [780]:
df_succ = df4[df4['status'] == 'success']
df_run = df4[df4['status'] != 'success']

In [782]:
su_train, su_test = train_test_split(df_succ, test_size=0.2, random_state=42)

In [783]:
ysu_train = su_train['money']
ysu_test = su_test['money'] 
Xsu_train = su_train.drop(['pledge', 'href', 'title', 'tags', 'status', 'tags_null', 'money'], axis=1)
Xsu_test = su_test.drop(['pledge', 'href', 'title', 'tags', 'status', 'tags_null', 'money'], axis=1)

Мы включиили текстовые данные в модель с помошью TF-IDF.

In [784]:
vec = TfidfVectorizer(min_df=0.001)
vec_train = vec.fit_transform(Xsu_train['description'])
vec_test = vec.transform(Xsu_test['description'])

scaler = MaxAbsScaler()
vec_train = scaler.fit_transform(vec_train)
vec_test = scaler.transform(vec_test)

In [785]:
vec_test.toarray()
vec_train.toarray()
vec.get_feature_names_out()

test_words = pd.DataFrame(vec_test.toarray(), columns=vec.get_feature_names_out())
train_words = pd.DataFrame(vec_train.toarray(), columns=vec.get_feature_names_out())

In [786]:
Xsu_train.reset_index(drop= True, inplace = True)
Xsu_test.reset_index(drop= True, inplace = True)

Xsu_train.rename(columns={"location": "loc"}, inplace = True)
Xsu_test.rename(columns={"location": "loc"}, inplace =True)

Xsu_tr_words = Xsu_train.join(train_words)
Xsu_test_words = Xsu_test.join(test_words)

In [787]:
numeric_features = ['upd', 'comm', 'supp', 'rewards_num', 'sentiment_score', 'ps_capital','inn_score', 'eco_score']
categorical_features = ['loc']

Определим трансформацию данных.

In [788]:
column_transformer = ColumnTransformer([
    ('scaling', StandardScaler(), numeric_features),
    ('ohe', OneHotEncoder(drop = 'first', handle_unknown = 'ignore'), categorical_features)
])

Создадаим пайплайн моделей и посмотрим, какие лучше решают нашу задачу.

In [ ]:
model_pipeline = [ 
    KNeighborsRegressor(),
    AdaBoostRegressor(),
    BaggingRegressor(),
    RandomForestRegressor(),
    ElasticNet(),
    MLPRegressor(),
    LinearSVR(),
    CatBoostRegressor(),
    XGBRegressor(),
    LGBMRegressor()
] # модели

MAPE_scores = [] # запоминаем скор каждой модели
MAE_scores = [] # запоминаем скор каждой модели
r2_scores = [] # запоминаем скор каждой модели

model_list = ['KNeighborsRegressor', 'AdaBoostRegressor', 'BaggingRegressor',
              'RandomForestRegressor', 'ElasticNet', 'MLPRegressor', 'LinearSVR',
             'CatBoostRegressor', 'XGBRegressor', 'LGBMRegressor']

for model in model_pipeline:
    simple_pipeline = Pipeline(steps=[
    ('ohe_and_scaling', column_transformer),
    ('regressor', model)])
    
    model_reg = simple_pipeline.fit(Xsu_tr_words, ysu_train)
    y_pred = model_reg.predict(Xsu_test_words)
    MAPE = mean_absolute_percentage_error(ysu_test, y_pred)
    r2 = r2_score(ysu_test, y_pred)
    MAE = mean_absolute_error(ysu_test, y_pred)
    MAPE_scores.append(MAPE)
    r2_scores.append(r2)
    MAE_scores.append(MAE)
    
 # тут прошла оптимизация, но мы скрыли страшный вывод

Мы рассмотрели несколько разных метрик: MAE (средняя абсолютная ошибка), MAPE (средняя абсолютная процентная ошибка) и R-квадрат. Мы выбрали те модели, которые показывают хороший результат сразу по нескольким метрикам, в основном R-квадрат, так как мы пытаемся найти модель, которая хорошо объясняет дисперсию зависимой переменной.

In [742]:
df_result = pd.DataFrame({'model': model_list, 'MAPE_score': MAPE_scores, 'R2': r2_scores, 'MAE': MAE_scores})
df_result.sort_values(by = 'R2')

,model,MAPE_score,R2,MAE
5,MLPRegressor,0.969230,-4.130071,29137.836460
6,LinearSVR,0.935254,-4.002528,28704.390050
4,ElasticNet,6.421012,0.171132,9899.082206
0,KNeighborsRegressor,4.201361,0.292181,9170.654302
1,AdaBoostRegressor,3.654899,0.318640,10005.407998
2,BaggingRegressor,1.137208,0.329427,8439.308862
8,XGBRegressor,1.160553,0.381505,8317.213593
9,LGBMRegressor,1.362276,0.406949,8100.839932
3,RandomForestRegressor,1.004238,0.425467,7689.492832
7,CatBoostRegressor,1.355528,0.444027,7923.467657


Лучше всего работают модели, основанные на решающих деревьях, в основном с бустингом.

Рассмотрим Random Forest.

In [633]:
# с помощью RandomizedSearchCV можно понять, какой примерно порядок переменных использовать
from sklearn.model_selection import RandomizedSearchCV 
n_estimators = [int(x) for x in np.linspace(start = 5, stop = 200, num = 10)]
max_features = ['log2', 'sqrt']
max_features.append(None)
max_depth = [int(x) for x in np.linspace(5, 100, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 8]
min_samples_leaf = [1, 2, 4]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,
                               scoring ='neg_mean_absolute_percentage_error' , n_iter = 50, cv = 3, verbose=1, random_state=42)
    
rf_random.fit(column_transformer.fit_transform(Xsu_tr_words), ysu_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=50,
                   param_distributions={'max_depth': [5, 14, 24, 33, 43, 52, 62,
                                                      71, 81, 90, 100, None],
                                        'max_features': ['log2', 'sqrt', None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 8],
                                        'n_estimators': [5, 26, 48, 70, 91, 113,
                                                         135, 156, 178, 200]},
                   random_state=42,
                   scoring='neg_mean_absolute_percentage_error', verbose=1)

In [634]:
rf_random.best_params_

{'n_estimators': 5,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': None,
 'max_depth': 100}

In [732]:
rf_tuned = RandomForestRegressor(n_estimators = 5, min_samples_split = 5, max_depth = 100, random_state = 1000)
simple_pipeline = Pipeline(steps=[
    ('ohe_and_scaling', column_transformer),
    ('regression_rf', rf_tuned)
])

model_reg = simple_pipeline.fit(Xsu_tr_words, ysu_train)
y_pred = model_reg.predict(Xsu_test_words)
y_train_pred = model_reg.predict(Xsu_tr_words)

print("Test MAPE = %.4f" % mean_absolute_percentage_error(ysu_test, y_pred))
print("Train MAPE = %.4f" % mean_absolute_percentage_error(ysu_train, y_train_pred))
print("Test MAE = %.4f" % mean_absolute_error(ysu_test, y_pred))
print("Train MAE = %.4f" % mean_absolute_error(ysu_train, y_train_pred))
print("Test R2 = %.4f" % r2_score(ysu_test, y_pred))
print("Train R2 = %.4f" % r2_score(ysu_train, y_train_pred))

Test MAPE = 1.3013
Train MAPE = 0.5569
Test MAE = 7834.5275
Train MAE = 4104.3615
Test R2 = 0.3817
Train R2 = 0.8116


Возможно, модель переобучается, так как на трейне результаты заметно получше, хотя кол-во деревьев и максимальная глубина при подборе параметров получились не очень большими. Средняя таргета 27 000, поэтому MAE = 7834.5275 не такая уж большая. Модель относительно неплохо объясняет таргет.

LinearSVR (посмотрим, насколько плохо справится модель даже с гиперпараметрами).

In [611]:
# оптимизируем с представлением о параметрах
from sklearn.model_selection import GridSearchCV 
# parameter grid
parameters_SVR = {'tol': [1e-4, 1e-3, 1e-5],
                 'C': [0.001, 0.01, 0.1, 1, 10, 100],
                 'loss': ['epsilon_insensitive', 'squared_epsilon_insensitive'],
                 'intercept_scaling': [1, 10]}
SVR = LinearSVR(random_state = 42)
reg_LinearSVR = GridSearchCV(SVR,                  
                   param_grid = parameters_SVR,   
                   scoring='neg_mean_absolute_error',      
                   cv=4,
                   verbose = 1)

reg_LinearSVR.fit(column_transformer.fit_transform(Xsu_tr_words), ysu_train)

Fitting 4 folds for each of 72 candidates, totalling 288 fits


GridSearchCV(cv=4, estimator=LinearSVR(random_state=42),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'intercept_scaling': [1, 10],
                         'loss': ['epsilon_insensitive',
                                  'squared_epsilon_insensitive'],
                         'tol': [0.0001, 0.001, 1e-05]},
             scoring='neg_mean_absolute_error', verbose=1)

In [612]:
reg_LinearSVR.best_params_

{'C': 100,
 'intercept_scaling': 10,
 'loss': 'epsilon_insensitive',
 'tol': 0.0001}

In [710]:
SVR_tuned = LinearSVR(C = 100, intercept_scaling = 10, tol = 0.0001)
simple_pipeline = Pipeline(steps=[
    ('ohe_and_scaling', column_transformer),
    ('regression_ridge', SVR_tuned)
])

model_reg = simple_pipeline.fit(Xsu_tr_words, ysu_train)
y_pred = model_reg.predict(Xsu_test_words)
y_train_pred = model_reg.predict(Xsu_tr_words)

print("Test MAPE = %.4f" % mean_absolute_percentage_error(ysu_test, y_pred))
print("Train MAPE = %.4f" % mean_absolute_percentage_error(ysu_train, y_train_pred))
print("Test MAE = %.4f" % mean_absolute_error(ysu_test, y_pred))
print("Train MAE = %.4f" % mean_absolute_error(ysu_train, y_train_pred))
print("Test R2 = %.4f" % r2_score(ysu_test, y_pred))
print("Train R2 = %.4f" % r2_score(ysu_train, y_train_pred))

Test MAPE = 6.7137
Train MAPE = 4.5929
Test MAE = 10020.6566
Train MAE = 9621.0546
Test R2 = 0.1275
Train R2 = 0.1451


У другой модели существенно выше MAE и MAPE на тесте, а предсказательная сила меньше.

Наконец, возьмём модель с бустингом.

In [789]:
X_train_opt = Xsu_tr_words.iloc[0:524] 
y_train_opt = ysu_train.iloc[0:524]
X_eval = Xsu_tr_words.iloc[524:] # достаём из трейна валидационную выборку
y_eval = ysu_train.iloc[524:]

In [790]:
categorical_features

['loc']

In [791]:
import optuna

In [792]:
def objective(trial):
    """Define the objective function"""

    params = {
        'tree_method':'approx',
        'max_depth': trial.suggest_int('max_depth', 1, 40),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 10, 120),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        'use_label_encoder': False,
        'random_state': 42
    }

    # Fit the model
    optuna_model = XGBRegressor(**params)
    simple_pipeline = Pipeline(steps=[
    ('ohe_and_scaling', column_transformer),
    ('regression_ridge', optuna_model)])

    model_reg = simple_pipeline.fit(X_train_opt, y_train_opt)

    # Make predictions
    y_pred = model_reg.predict(X_eval)

    # Evaluate predictions
    r2 = r2_score(y_eval, y_pred)
    return r2

In [793]:
study = optuna.create_study(direction='maximize')

[I 2023-06-14 18:40:13,925] A new study created in memory with name: no-name-9a281546-332c-42f5-aec5-2eb87808df8f


In [ ]:
study.optimize(objective, n_trials=200) # тут прошла оптимизация, но мы скрыли страшный вывод

In [795]:
trial = study.best_trial
params = trial.params

model = XGBRegressor(**params)

simple_pipeline = Pipeline(steps=[
    ('ohe_and_scaling', column_transformer),
    ('regression_ridge', model)])

model_reg = simple_pipeline.fit(Xsu_tr_words, ysu_train)

In [796]:
y_pred = model_reg.predict(Xsu_test_words)
y_train_pred = model_reg.predict(Xsu_tr_words)

print("Test MAPE = %.4f" % mean_absolute_percentage_error(ysu_test, y_pred))
print("Train MAPE = %.4f" % mean_absolute_percentage_error(ysu_train, y_train_pred))
print("Test MAE = %.4f" % mean_absolute_error(ysu_test, y_pred))
print("Train MAE = %.4f" % mean_absolute_error(ysu_train, y_train_pred))
print("Test R2 = %.4f" % r2_score(ysu_test, y_pred))
print("Train R2 = %.4f" % r2_score(ysu_train, y_train_pred))

Test MAPE = 1.1020
Train MAPE = 0.2497
Test MAE = 7615.4878
Train MAE = 1634.5163
Test R2 = 0.4862
Train R2 = 0.9764


MAE на тесте немного меньше MAE случайного леса, R-квадрат близок к 0.5.

**Итог**: самой лучшей моделью оказался XGBRegressor, далее идёт регрессия с помощью случайного леса. Лучшие метрики, полученные на тесте MAE = 7615.4878, R2 = 0.4862. Мы пытаемся предсказать поведение людей в социальной ситуации, поэтому не стоит рассчитывать на коэффициент детерминации выше 0.6.

Мы попробовали реализовать предсказание среднего взноса по всем данным, но максимальный полученный Test R2 для XGB регрессора не превышал 0.2. Средний взнос был получен как частное денег и кол-ва спонсоров. Действительно, даже теоретически не совсем понятно, какие факторы определяют эту переменную. Возможно, она зависит от предпочтений и особенностей конкретного человека.